# Devoir maison pour le compte du cours de DBS

## 1 Choix des jeux de données

### Les sources de données

1) NBA games data, Nathan Lugan (Kaggle)
Link : https://www.kaggle.com/datasets/nathanlauga/nba-games?select=games.csv

Colonnes d'intéret : 

GAME_DATE_EST : Le jour du match

GAME_ID : L'identifiant du match

HOME_TEAM_ID : L'identifiant de l'équipe à domicile

VISITOR_TEAM_ID : L'identifiant de l'équipe à l'exterieur

SEASON : La saison

PTS_home : Nombre de points dans le match de l'équipe à domicile

PTS_away : Nombre de points dans le match de l'équipe à l'exterieur

HOME_TEAM_WINS : Valeur booléenne qui si l'équipe à domicile à gagné le match (1) ou non (0)

In [1]:
"""
2) NBA Dataset - Box Scores & Stats, 1947 - Today, Justinas Cirtautas (Kaggle). Link : https://www.kaggle.com/datasets/eoinamoore/historical-nba-data-and-player-box-scores

gameId : Identifant du match
hometeamCity : Ville de l'équipe à domicile
hometeamName : Nom de l'équipe à l'exterieur
hometeamId : Identifiant de l'équipe à domicile
awayteamCity : Ville de l'équipe à l'exterieur
awayteamName : Nom de l'équipe à l'exterieur
awayteamId : Identifiant de l'équipe à l'exterieur

"""

"\n2) NBA Dataset - Box Scores & Stats, 1947 - Today, Justinas Cirtautas (Kaggle). Link : https://www.kaggle.com/datasets/eoinamoore/historical-nba-data-and-player-box-scores\n\ngameId : Identifant du match\nhometeamCity : Ville de l'équipe à domicile\nhometeamName : Nom de l'équipe à l'exterieur\nhometeamId : Identifiant de l'équipe à domicile\nawayteamCity : Ville de l'équipe à l'exterieur\nawayteamName : Nom de l'équipe à l'exterieur\nawayteamId : Identifiant de l'équipe à l'exterieur\n\n"

3) NBA Database, Wyatt Walsh (Kaggle). Lien : https://www.kaggle.com/datasets/wyattowalsh/basketball

    person_id : Identifiant de chaque joueur

    first_name : Prénom du joueur

    last_name : Nom du joueur

    birthdate : Date d'anniversaire

    school : L'université de laquelle le joueur vient

    height : La taille du joueur

    position : Sa position

    team_id : L'identifiant de son équipe

    team_name : Le nom de son équipe

    team_city : Ville de son équipe

    team_abbreviation : L'abréviation de son équipe

    draft_year : L'année de draft du joueur

    draft_round : Le tour auquel le joueur a été drafté



# II. Passe au code

In [2]:
import pandas as pd

In [18]:
# Import the dataset of players

players = pd.read_csv("./csv_datasets/Players.csv")
players = players[["person_id", "first_name", "last_name", "birthdate", "school", "height", "position", "team_id", "team_name", "team_city", "team_abbreviation", "draft_year", "draft_round"]]
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4171 entries, 0 to 4170
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   person_id          4171 non-null   int64 
 1   first_name         4171 non-null   object
 2   last_name          4171 non-null   object
 3   birthdate          4171 non-null   object
 4   school             4156 non-null   object
 5   height             4075 non-null   object
 6   position           4108 non-null   object
 7   team_id            4171 non-null   int64 
 8   team_name          3469 non-null   object
 9   team_city          3469 non-null   object
 10  team_abbreviation  3469 non-null   object
 11  draft_year         4171 non-null   object
 12  draft_round        4007 non-null   object
dtypes: int64(2), object(11)
memory usage: 423.7+ KB


In [17]:
# Import the Dataset games
games = pd.read_csv("./csv_datasets/Games.csv")
games = games[["GAME_DATE_EST", "GAME_ID", "HOME_TEAM_ID", "VISITOR_TEAM_ID", "SEASON", "PTS_home", "PTS_away", "HOME_TEAM_WINS"]]
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26651 entries, 0 to 26650
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   GAME_DATE_EST    26651 non-null  object 
 1   GAME_ID          26651 non-null  int64  
 2   HOME_TEAM_ID     26651 non-null  int64  
 3   VISITOR_TEAM_ID  26651 non-null  int64  
 4   SEASON           26651 non-null  int64  
 5   PTS_home         26552 non-null  float64
 6   PTS_away         26552 non-null  float64
 7   HOME_TEAM_WINS   26651 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 1.6+ MB


In [ ]:
Game

# Merge both dataset

nba_df = games.merge(players, on=")